In [44]:
import chaospy as cp 
import numpy as np 
import matplotlib.pyplot as plt

In [22]:
import numpy as np
from scipy.integrate import odeint
from y_ODE import *
from z_ODE import *
from DH_dy import * 
from DH_dz import * 
from theta_ODE import *
from fi_ODE import *
from utilities import *
import matplotlib.pyplot as plt 
from check_constraint import *
import math


   
def model(kg,Eg,g,kb,Eb,b):

    delta_t = 1
    t0 = 0
    tf = 1800  ## batch_time

    parameters = {}
    parameters["kg"] = kg
    parameters["Eg"] = Eg 
    parameters["g"] = g
    parameters["kb"] = kb
    parameters["Eb"] = Eb
    parameters["b"] = b
    parameters["rho"] = 2.66*10**-12;
    parameters["kv"] = 0.54
    #rho = parameters["rho"]


    y0 = np.array([0.1743,66.66,1.83*10**4,5.05*10**6,1.93*10**9,0.867,0,0,0])
    y0 = y0.reshape(1,-1)
    zf = np.array([0,0,0,0,1,0,0,0,-1])
    zf = zf.reshape(1,-1)
    theta0 = np.array([0,0,0,0,0,0,0,0,0])
    theta0 = theta0.reshape(1,-1)
    fi_f = np.array([0,0,0,0,0,0,0,0,0])
    fi_f = fi_f.reshape(1,-1)

    M = -10**-6
    tolerance = 10**-2

    num_iter = 1

    time_length = len(range(t0,tf+delta_t,delta_t))
    T_vec = np.ones(time_length)*323
    DH_vec = np.zeros((num_iter,time_length))



    iteration = 0


    print y0.shape


    while(iteration < num_iter) :


        y_mat = np.zeros((time_length,9))
        z_mat = np.zeros((time_length,9))
        theta_mat = np.zeros((time_length,9))
        fi_mat = np.zeros((time_length,9))

        DelH_dy_mat = np.zeros((time_length,9))
        DelH_dz_mat = np.zeros((time_length,9))

        y_mat[0,:] = y0
        #print y_mat[0,0]
        z_mat[0,:] = zf
        theta_mat[0,:] = theta0
        fi_mat[0,:] = fi_f


        print "y forward integration"
        for t in range(t0,tf,delta_t) :

            t_horizon = np.linspace(t,t+delta_t,num = 10)
            T = T_vec[t]
            C = y_mat[t,0]
            G = calG(T,C,parameters)
            B = calB(y_mat[t,:],T,parameters)


            y = odeint(y_ODE,y_mat[t,:],t_horizon,args = (T,C,G,B,parameters))

            y_mat[t+delta_t,:] = y[-1,:]

        print "Z backward ..."
        for t in range(t0,tf,delta_t):

            T = T_vec[t]
            C = y_mat[t,0]
            G = calG(T,C,parameters)

            t_horizon = np.linspace(t,t+delta_t,num = 10)
            z = odeint(z_ODE,z_mat[t,:],t_horizon,args = (G,parameters,T,y_mat[t,:]))
            z_mat[t+delta_t,:] = z[-1,:]

        print "DH .."
        for t in range(t0,tf+delta_t,delta_t):


            T = T_vec[t]
            G = calG(T,C,parameters)

            DelH_dy_mat[t,:] = DH_dy(y_mat[t,:],z_mat[t,:],G,T,parameters)
            DelH_dz_mat[t,:] = DH_dz(T,y_mat[t,:],parameters)

        print " Theta forward integration.."
        for t in range(t0,tf,delta_t) :

            T = T_vec[t]
            t_horizon = np.linspace(t,t+delta_t,num = 10)
            theta = odeint(theta_ODE,theta_mat[t,:],t_horizon,args = (y_mat[t,:],T,parameters))

            theta_mat[t+delta_t,:] = theta[-1,:]

        print "Fi backward .."
        for t in range(t0,tf,delta_t):

            T = T_vec[t]
            t_horizon = np.linspace(t,t+delta_t,num = 10)


            fi = odeint(fi_ODE,fi_mat[t,:],t_horizon,args = (y_mat[t,:],z_mat[t,:],theta_mat[t,:],T,parameters))
            #print z[-1,0]			
            fi_mat[t+delta_t,:] = fi[-1,:]

        print "Derivative sums...."
        for t in range(t0,tf+delta_t,delta_t) :
            var_sum = 0 

            for i in range(9):
                var_sum +=  DelH_dy_mat[t,i]*theta_mat[t,i] + DelH_dz_mat[t,i]*fi_mat[t,i] 
                ## + +  
            DH_vec[iteration,t] = var_sum

        print "check constraints...."
        for t in range(t0,tf+delta_t,delta_t) :

            if abs(DH_vec[iteration,t]) > tolerance :

                #print "Here"
                T_vec[t] = check_constraint(T_vec[t],y_mat[t,0],DH_vec[iteration,t],M)


        plt_1 =  DH_vec[iteration,:]


        ## Plotting function 
        """
        t = np.linspace(t0,tf,num = 1801)
        plt.figure(0)
        plt.plot(t,plt_1,'b')
        #plt.show()
        plt.figure(1)
        plt.plot(t,T_vec)
        plt.figure(2)
        plt.plot(t,y_mat[:,4]-y_mat[:,8])
        plt.show()
        plt.cla()

        """

        iteration+=1



    return y_mat

In [18]:
cp.seed(1)

dist_kg = cp.Uniform(1.368*10**8,1.512*10**8)
dist_Eg = cp.Uniform(4616,5101)
dist_g = cp.Uniform(1.425,1.575)
dist_kb = cp.Uniform(270,299)
dist_Eb = cp.Uniform(7141,7892)
dist_b = cp.Uniform(1.3775,1.5225)

In [32]:
dist = cp.J(dist_kg,dist_Eg,dist_g,dist_kb,dist_Eb,dist_b)
P = cp.orth_ttr(1,dist)
nodes , weights = cp. generate_quadrature (2, dist)


In [33]:
len(nodes[0])

729

sanples_u = []


In [34]:
samples_u = []


In [35]:
i = 0 

for node in nodes.T:
    print i 
    samples_u.append(model(node[0],node[1],node[2],node[3],node[4],node[5]))
    
    i+=1

0
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
1
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
2
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
3
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
4
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
5
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
6
(1, 9)
y forward integration
Z backward ...
DH ..
 Theta forward integration..
Fi backward ..
Derivative sums....
check constraints....
7
(1, 9)
y forward integration
Z b

In [67]:
samples_u[1][1800]

for i in range(729):
    z_samples = samples_u[i][:]
    z_obj = [i[4] - i[8] for i in z_samples]
    ##z_9 = [i[8] for i in z_samples]
    plt.plot(range(0,1801,1),z_obj)
    
plt.show()

In [59]:
z =samples_u[0][:]
len(z)

1801

In [63]:
z_C = [i[0] for i in z]
plt.plot(range(0,1801,1),z_C)
plt.show()

In [69]:
U_hat = cp.fit_quadrature(P, nodes, weights, samples_u)

In [71]:
mean = cp.E(U_hat,dist)

In [84]:
z = [i[4] - i[8] for i in mean]

plt.plot(range(0,1801,1),z)
plt.show()

In [83]:
print U_hat[1][0]

-5.09645500594e-14q0+2.26822312187e-08q1+2.41568387264e-05q2-3.98962549605e-17q3+3.5099382173e-17q4+3.76365745063e-14q5+0.174153574074
